In [58]:
import os
import json
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
import xgboost as xgb

path=!pwd
SCRIPT_DIR = os.path.realpath(path[0])
print(SCRIPT_DIR)

/home/etsu/projects/Data_Mining_in_Action_2018_Spring/industry/hw02


In [47]:
%matplotlib notebook

In [3]:
data_path=SCRIPT_DIR + '/../seminar02/HR.csv'
df = pd.read_csv(data_path)
target_ = 'left'
features_ = [c for c in df if c != target_]
target = np.array(df[target_])
data = np.array(df[features_])

In [33]:
unique, counts = np.unique(target, return_counts=True)
unique, counts

(array([0, 1]), array([9821, 5178]))

In [57]:
df.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [11]:
params = {
    "learning_rate": 0.1, 
    "max_depth": 5, 
    "n_estimators": 1000, 
    "min_child_weight": 3,
    "seed": 42
}

# max_depth=3
# learning_rate=0.1
# n_estimators=100
# objective='binary:logistic'
# booster='gbtree'
# n_jobs=3
# gamma=0
# min_child_weight=1
# max_delta_step=0
# subsample=1
# colsample_bytree=1
# colsample_bylevel=1
# reg_alpha=0
# reg_lambda=1
# scale_pos_weight=1
# base_score=0.5
# random_state=0
# seed=None
# missing=None

In [26]:
%%time
estimator = xgb.XGBClassifier()
baseline_score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Baseline score {}'.format(baseline_score))

Baseline score 0.779121224244849
CPU times: user 1.34 s, sys: 16 ms, total: 1.35 s
Wall time: 1.44 s


In [27]:
%%time
params = {
    "learning_rate": 0.1, 
    "max_depth": 5, 
    "n_estimators": 1000, 
    "min_child_weight": 3,
    "seed": 42
}
estimator = xgb.XGBClassifier(**params)
score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Default score {} ({})'.format(score, score-baseline_score))

Default score 0.7659867306794692 (-0.013134493565379768)
CPU times: user 20.8 s, sys: 180 ms, total: 21 s
Wall time: 21.2 s


In [118]:
%%time
param_grid1 = {
    "n_estimators": range(50,90,1)
}

xgb1 = xgb.XGBClassifier(
    learning_rate=0.1,
    seed=42
)

grid1 = GridSearchCV(xgb1, param_grid1, scoring='accuracy', n_jobs=4, cv=3)
grid1.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid1.best_params_,
          grid1.best_score_, grid1.best_score_ - baseline_score
      ))

=====
Best params {'n_estimators': 82}
Score 0.7796519767984532 (0.0005307525536042412)
=====
CPU times: user 1.32 s, sys: 712 ms, total: 2.03 s
Wall time: 18.7 s


In [85]:
%%time
param_grid2 = {
    'max_depth':range(1,6),
    'min_child_weight':range(1,6)
}

xgb2 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    seed=42
)

grid2 = GridSearchCV(xgb2, param_grid2, scoring='accuracy', n_jobs=3, cv=3)
grid2.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid2.best_params_,
          grid2.best_score_, grid2.best_score_ - baseline_score
      ))

=====
Best params {'max_depth': 5, 'min_child_weight': 3}
Score 0.7802520168011201 (0.0011307925562711318)
=====
CPU times: user 1.16 s, sys: 424 ms, total: 1.59 s
Wall time: 13.2 s


In [88]:
%%time
param_grid3 = {
    'gamma':[i/100.0 for i in range(0,5)]
}

xgb3 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    seed=42
)

grid3 = GridSearchCV(xgb3, param_grid3, scoring='accuracy', n_jobs=3, cv=3)
grid3.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid3.best_params_,
          grid3.best_score_, grid3.best_score_ - baseline_score
      ))

=====
Best params {'gamma': 0.01}
Score 0.7803186879125275 (0.0011974636676784778)
=====
CPU times: user 820 ms, sys: 184 ms, total: 1 s
Wall time: 5.37 s


In [91]:
%%time
param_grid4 = {
    'subsample':[i/100.0 for i in range(85,100)],
    'colsample_bytree':[i/100.0 for i in range(75,85)]
}

xgb4 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    gamma=0.1,
    seed=42
)

grid4 = GridSearchCV(xgb4, param_grid4, scoring='accuracy', n_jobs=3, cv=3)
grid4.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid4.best_params_,
          grid4.best_score_, grid4.best_score_ - baseline_score
      ))

=====
Best params {'colsample_bytree': 0.75, 'subsample': 0.97}
Score 0.7818521234748983 (0.0011974636676784778)
=====
CPU times: user 2.92 s, sys: 2.37 s, total: 5.29 s
Wall time: 1min 53s


In [95]:
%%time
param_grid5 = {
    'reg_alpha':[0, 1e-10, 1e-5, 1e-2]
}

xgb5 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    gamma=0.1,
    colsample_bytree=0.75,
    subsample=0.97,
    seed=42
)

grid5 = GridSearchCV(xgb5, param_grid5, scoring='accuracy', n_jobs=3, cv=3)
grid5.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid5.best_params_,
          grid5.best_score_, grid5.best_score_ - baseline_score
      ))

=====
Best params {'reg_alpha': 0}
Score 0.7818521234748983 (0.0027308992300493218)
=====
CPU times: user 840 ms, sys: 116 ms, total: 956 ms
Wall time: 3.76 s


In [139]:
delta = -0.0003
max(round(2 ** (30 * delta), 2), 0.05)

0.99

In [126]:
%%time
param_grid6 = {
    'n_estimators': range(62,65),
    'max_depth': [6,7,8,9,10],
    'min_child_weight': [2,3,4],
}

xgb6 = xgb.XGBClassifier(
    learning_rate=0.1,
    gamma=0.1,
    colsample_bytree=0.75,
    subsample=0.97,
    seed=42
)

grid6 = GridSearchCV(xgb6, param_grid6, scoring='accuracy', n_jobs=3, cv=3)
grid6.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid6.best_params_,
          grid6.best_score_, grid6.best_score_ - baseline_score
      ))

=====
Best params {'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 62}
Score 0.7827188479231949 (0.0035976236783459292)
=====
CPU times: user 2.04 s, sys: 1.4 s, total: 3.44 s
Wall time: 59.4 s


In [141]:
%%time
params = {
    "learning_rate": 0.1,
    "n_estimators": 62,
    "max_depth": 7, 
    "min_child_weight": 4,
    "gamma": 0.1,
    "colsample_bytree": 0.75,
    "subsample": 0.97,
    "seed": 42
}
estimator = xgb.XGBClassifier(**params)
score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Score {} ({})'.format(score, score-baseline_score))

Default score 0.7827216643328666 (0.0036004400880176313)
CPU times: user 1.52 s, sys: 64 ms, total: 1.58 s
Wall time: 1.67 s
